In [1]:
# !pip install selenium
# !pip install webdriver-manager
# View your Google Chrome browser version: chrome://settings/help
# Download ChromeDriver that corresponds to your Google Chrome browser version: https://sites.google.com/chromium.org/driver/

Useful links:
https://stackoverflow.com/questions/69875125/find-element-by-commands-are-deprecated-in-selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [3]:
CHROME_DRIVER_PATH = "C:/Users/ernes/Downloads/chromedriver_win32/chromedriver.exe"
DATA_WRITE_PATH = "C:/Users/ernes/Git/dsa3101-2220-12-ds/Backend/Data/jobs/"

In [4]:
ser = Service(CHROME_DRIVER_PATH)
op = webdriver.ChromeOptions()

# JobStreet Scraping

In [5]:
website = "https://www.jobstreet.com.sg/"
driver = webdriver.Chrome(service=ser, options=op)
driver.get(website)

In [6]:
query = '"data science"'
all_jobs_links = []

In [7]:
# Finding search bar
search = driver.find_element(By.ID, "searchKeywordsField")

# Remove all words in search bar
search.send_keys(Keys.CONTROL + "a")
search.send_keys(Keys.DELETE)

# Send query
search.send_keys(query)
search.send_keys(Keys.RETURN)

# Waiting for 10 seconds after sending keys to search bar
try:
    # Finding element by XPATH and class attribute since div class names are dynamic
    jobs = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//*[@data-automation='jobListing']"))
    )
    
except:
    driver.quit()

In [8]:
next_page_url = []
visited = []
queue = [driver.current_url]

In [ ]:
start_time = time.time()
while len(queue) > 0: # REVISE
    
    # Visiting all pages
    
    # Dequeue next URL
    next_url = queue[-1]
    queue = queue[:-1]
    visited.append(next_url)
    print(f"Visiting {next_url}")
    driver.get(next_url)
    
    # Waiting for 10 seconds after sending keys to search bar
    try:
        # Finding element by XPATH and class attribute since div class names are dynamic
        jobs = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@data-automation='jobListing']"))
        )
        
        # Getting the job postings on the page
        jobs_list = jobs.find_elements(By.XPATH, "./*")[:-2] # The last two results are invalid

        # Getting the links for each job posting
        jobs_links_found = [job.find_element(By.TAG_NAME, "h1").find_element(By.XPATH, "a[@href]").get_attribute("href") for job in jobs_list]
        all_jobs_links.extend(jobs_links_found)
        
    except:
        print(f"FAILED: {next_url}")
        driver.quit()
    
    # Finding new URLs
    next_page_url = [page.get_attribute("href") for page in driver.find_element(By.XPATH, "//*[@data-automation='pagination']").find_elements(By.XPATH, "a[@href]")]
    for url in next_page_url:
        if url not in visited:
            queue.append(url)
            
print(f"Job completed in {round(time.time() - start_time)} seconds")

Visiting https://www.jobstreet.com.sg/%22data-science%22-jobs
Visiting https://www.jobstreet.com.sg/%22data-science%22-jobs?pg=2
Visiting https://www.jobstreet.com.sg/%22data-science%22-jobs?pg=3
Visiting https://www.jobstreet.com.sg/%22data-science%22-jobs?pg=4
Visiting https://www.jobstreet.com.sg/%22data-science%22-jobs?pg=5
Visiting https://www.jobstreet.com.sg/%22data-science%22-jobs?pg=6
Visiting https://www.jobstreet.com.sg/%22data-science%22-jobs?pg=7
Visiting https://www.jobstreet.com.sg/%22data-science%22-jobs?pg=8
Visiting https://www.jobstreet.com.sg/%22data-science%22-jobs?pg=9
Visiting https://www.jobstreet.com.sg/%22data-science%22-jobs?pg=10
Visiting https://www.jobstreet.com.sg/%22data-science%22-jobs?pg=11
Visiting https://www.jobstreet.com.sg/%22data-science%22-jobs?pg=12
Visiting https://www.jobstreet.com.sg/%22data-science%22-jobs?pg=13
Visiting https://www.jobstreet.com.sg/%22data-science%22-jobs?pg=14
Visiting https://www.jobstreet.com.sg/%22data-science%22-jobs?

In [ ]:
driver.quit()

In [ ]:
print(f"{len(all_jobs_links)} job sites found")
print(all_jobs_links[:5])

In [ ]:
with open(DATA_WRITE_PATH + 'jobstreet_job_url_query-data_science.txt', 'w') as f:
    for url in all_jobs_links:
        f.write(url)
        f.write("\n")